In [99]:
#Import dependencies
import pandas as pd
import os

In [100]:
#Read CSV file
pymoli_data = os.path.join("Resources", "purchase_data.json.txt")
pymoli_df = pd.read_json(pymoli_data)

In [101]:
#Print first five rows of data
pymoli_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [102]:
#Count number of unique players
players = pymoli_df["SN"].nunique()
players

573

In [103]:
#Create dataframe for number of players
players_df = pd.DataFrame([{'Total Players': players}])
players_df

,Total Players
0,573


In [104]:
#Purchasing Analysis

In [105]:
#Count number of unique items
items = pymoli_df["Item ID"].nunique()
items

183

In [106]:
#Calculate average purchase price of items
avg_price = round(pymoli_df['Price'].mean(),2)
avg_price

2.93

In [107]:
#Count total number of purchases
purchases = pymoli_df["SN"].count()
purchases

780

In [108]:
#Calculate total revenue of item sales
revenue = round(pymoli_df["Price"].sum(),2)
revenue

2286.33

In [109]:
#Create dataframe for purchase data
purchase_analysis = pd.DataFrame([{'Number of Unique Items': items,
                                  'Average Purchase Price': avg_price,
                                  'Total Number of Purchases': purchases,
                                  'Total Revenue': revenue}])
purchase_analysis

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,2.93,183,780,2286.33


In [110]:
#Gender Demographics

In [111]:
#Create players dataframe without duplicate players
unique_players = pymoli_df.drop_duplicates(['SN'], keep ='last')
unique_players.head()

,Age,Gender,Item ID,Item Name,Price,SN
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97


In [112]:
#Count total players by gender
gender_total = unique_players["Gender"].value_counts().reset_index()
#Rename columns for clarity
gender_total.rename(columns = {'index': 'Gender', 'Gender': 'Total Count'}, inplace = True)
#Calculate percentage of players that fall into each gender category
gender_percent = gender_total["Total Count"]/players*100
gender_percent
#Add gender percentage column to dataframe
gender_total["Percentage of Players"] = gender_percent.round(2)
gender_total

,Gender,Total Count,Percentage of Players
0,Male,465,81.15
1,Female,100,17.45
2,Other / Non-Disclosed,8,1.40


In [113]:
#Purchasing Analysis - Gender

In [114]:
#Create dataframe for average price by gender
purchase_price_gender = pd.DataFrame(round(pymoli_df.groupby('Gender')['Price'].mean(),2))
#Rename and format price column
purchase_price_gender.rename(columns = {'Price': 'Average Purchase Price'}, inplace = True)
purchase_price_gender

,Average Purchase Price
Gender,
Female,2.82
Male,2.95
Other / Non-Disclosed,3.25


In [115]:
#Calculate total purchase value by gender
purchase_total_gender = pd.DataFrame(pymoli_df.groupby('Gender')['Price'].sum())
purchase_total_gender

,Price
Gender,
Female,382.91
Male,1867.68
Other / Non-Disclosed,35.74


In [116]:
#Rename column
purchase_total_gender.rename(columns = {'Price': 'Total Purchase Value'}, inplace = True)
purchase_total_gender

,Total Purchase Value
Gender,
Female,382.91
Male,1867.68
Other / Non-Disclosed,35.74


In [117]:
#Merge average price and purchase count dataframes
purchase_analysis_gender = pd.merge(purchase_price_gender, purchase_total_gender, left_index = True, right_index = True)
purchase_analysis_gender

,Average Purchase Price,Total Purchase Value
Gender,,
Female,2.82,382.91
Male,2.95,1867.68
Other / Non-Disclosed,3.25,35.74


In [118]:
#Count number of players in each gender category
players_by_gender = pd.DataFrame(pymoli_df.groupby('Gender')['SN'].count())
players_by_gender

,SN
Gender,
Female,136
Male,633
Other / Non-Disclosed,11


In [148]:
#Add column for total purchase value to previous dataframe
gender_purchase_analysis = pd.merge(purchase_analysis_gender, purchase_total_gender, left_index = True, right_index = True)
gender_purchase_analysis.rename(columns = {'Price': 'Average Purchase Price'}, inplace = True)
gender_purchase_analysis

,Average Purchase Price,Total Purchase Value_x,Total Purchase Value_y
Gender,,,
Female,2.82,382.91,382.91
Male,2.95,1867.68,1867.68
Other / Non-Disclosed,3.25,35.74,35.74


In [149]:
#Drop the duplicate column
del gender_purchase_analysis['Total Purchase Value_y']

In [151]:
#Rename column
gender_purchase_analysis.rename(columns = {'Total Purchase Value_x': 'Total Purchase Value'}, inplace = True)
gender_purchase_analysis

,Average Purchase Price,Total Purchase Value
Gender,,
Female,2.82,382.91
Male,2.95,1867.68
Other / Non-Disclosed,3.25,35.74


In [152]:
#Calculate normalized totals
normalized_totals = gender_purchase_analysis['Total Purchase Value']/players_by_gender['SN']
normalized_totals

Gender
Female                   2.815515
Male                     2.950521
Other / Non-Disclosed    3.249091
dtype: float64

In [153]:
#Add column for normalized totals 
gender_purchase_analysis["Normalized Totals"] = normalized_totals.round(2)
gender_purchase_analysis

,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,
Female,2.82,382.91,2.82
Male,2.95,1867.68,2.95
Other / Non-Disclosed,3.25,35.74,3.25


In [154]:
#Age Demographics

In [155]:
#Print minimum and maximum age values
print(pymoli_df["Age"].min())
print(pymoli_df["Age"].max())

7
45


In [156]:
#Create bins for age groups
age = [0,10,15,20,25,30,35,40,100]

# Create labels for these bins
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

In [157]:
#Create age range conditions
pymoli_df.loc[(pymoli_df['Age'] < 10), 'age'] = "0 - 10"
pymoli_df.loc[(pymoli_df['Age'] >= 10) & (pymoli_df['Age'] <= 14), 'age'] = "10 - 14"
pymoli_df.loc[(pymoli_df['Age'] >= 15) & (pymoli_df['Age'] <= 19), 'age'] = "15 - 19"
pymoli_df.loc[(pymoli_df['Age'] >= 20) & (pymoli_df['Age'] <= 24), 'age'] = "20 - 24"
pymoli_df.loc[(pymoli_df['Age'] >= 25) & (pymoli_df['Age'] <= 29), 'age'] = "25 - 29"
pymoli_df.loc[(pymoli_df['Age'] >= 30) & (pymoli_df['Age'] <= 34), 'age'] = "30 - 34"
pymoli_df.loc[(pymoli_df['Age'] >= 35) & (pymoli_df['Age'] <= 39), 'age'] = "35 - 39"
pymoli_df.loc[(pymoli_df['Age'] >= 40), 'age'] = "40+"

In [158]:
#Count number of purchases by users within each age range
age_purchase_count = pd.DataFrame(pymoli_df.groupby('age')['SN'].count())
#Rename column
age_purchase_count.rename(columns = {'SN': 'Purchase Count'}, inplace = True)
age_purchase_count

,Purchase Count
age,
0 - 10,28
10 - 14,35
15 - 19,133
20 - 24,336
25 - 29,125
30 - 34,64
35 - 39,42
40+,17


In [159]:
#Count number of unique players within each age range
age_purchase_count_unique = pd.DataFrame(pymoli_df.groupby('age')['SN'].nunique())
#Rename column
age_purchase_count_unique.rename(columns = {'SN': 'Number of Buyers'}, inplace = True)
age_purchase_count_unique

,Number of Buyers
age,
0 - 10,19
10 - 14,23
15 - 19,100
20 - 24,259
25 - 29,87
30 - 34,47
35 - 39,27
40+,11


In [160]:
#Calculate average purchase price within age ranges
age_average_price = pd.DataFrame(round(pymoli_df.groupby('age')['Price'].mean(),2))

In [161]:
#Calculate total purchase price within age ranges
age_total_value = pd.DataFrame(round(pymoli_df.groupby('age')['Price'].sum(),2))

In [162]:
#Create dataframe of purchase info
age_demographics = pd.merge(age_purchase_count, age_average_price, left_index = True, right_index = True)
age_demographics = pd.merge(age_demographics, age_total_value, left_index = True, right_index = True)
#Rename columns
age_demographics.rename(columns = {'Price_x': 'Average Purchase Price', 'Price_y': 'Total Purchase Value'}, inplace = True)
age_demographics

,Purchase Count,Average Purchase Price,Total Purchase Value
age,,,
0 - 10,28,2.98,83.46
10 - 14,35,2.77,96.95
15 - 19,133,2.91,386.42
20 - 24,336,2.91,978.77
25 - 29,125,2.96,370.33
30 - 34,64,3.08,197.25
35 - 39,42,2.84,119.40
40+,17,3.16,53.75


In [163]:
#Calculate normalized totals
age_normalized_totals = round(age_demographics['Total Purchase Value']/age_purchase_count_unique['Number of Buyers'],2)
age_normalized_totals

age
0 - 10     4.39
10 - 14    4.22
15 - 19    3.86
20 - 24    3.78
25 - 29    4.26
30 - 34    4.20
35 - 39    4.42
40+        4.89
dtype: float64

In [164]:
#Add column for normalized totals
age_demographics["Normalized Totals"] = age_normalized_totals
age_demographics

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
age,,,,
0 - 10,28,2.98,83.46,4.39
10 - 14,35,2.77,96.95,4.22
15 - 19,133,2.91,386.42,3.86
20 - 24,336,2.91,978.77,3.78
25 - 29,125,2.96,370.33,4.26
30 - 34,64,3.08,197.25,4.20
35 - 39,42,2.84,119.40,4.42
40+,17,3.16,53.75,4.89


In [165]:
#Top Spenders

In [166]:
#Calculate spending values for individual users
name_purchase_sum = pd.DataFrame(round(pymoli_df.groupby('SN')['Price'].sum(),2))
name_purchase_count = pd.DataFrame(round(pymoli_df.groupby('SN')['Price'].count(),2))
name_purchase_average = pd.DataFrame(round(pymoli_df.groupby('SN')['Price'].mean(),2))

In [167]:
#Convert values into dataframe
top_five = pd.merge(name_purchase_sum, name_purchase_count, left_index = True, right_index = True).merge(name_purchase_average, left_index=True, right_index=True)
#Rename columns
top_five.rename(columns = {'Price_x': 'Total Purchase Price', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)
#Sort by highest spenders
top_five.sort_values('Total Purchase Price', ascending = False, inplace=True)
top_five.head(n=5)

,Total Purchase Price,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,17.06,5,3.41
Saedue76,13.56,4,3.39
Mindimnya67,12.74,4,3.18
Haellysu29,12.73,3,4.24
Eoda93,11.58,3,3.86


In [168]:
#Most Popular Items

In [180]:
#Start a dataframe of item counts by Item ID
popular_items = pd.DataFrame(pymoli_df.groupby('Item ID')['Item ID'].count())
popular_items.head()
#Create dataframe by Item ID to hold price totals
popular_items_revenue = pd.DataFrame(pymoli_df.groupby('Item ID')['Price'].sum())
#Merge dataframes
prices = pd.merge(popular_items_revenue, popular_items, left_index=True, right_index=True)
#Rename column
prices.rename(columns = {"Item ID": "Purchase Count"}, inplace=True)
prices.head()

,Price,Purchase Count
Item ID,,
0,1.82,1
1,9.12,4
2,3.40,1
3,1.79,1
4,2.28,1


In [181]:
#Create dataframe for item names indexed by Item ID
popular_item_names = pd.DataFrame(pymoli_df["Item Name"])
#Merge it with pricing dataframe
names_with_prices = pd.merge(popular_item_names, prices, left_index=True, right_index=True)

In [187]:
#Calculate total purchase value and drop into new column
names_with_prices['Total Purchase Value'] = names_with_prices['Price'] * names_with_prices['Purchase Count']
names_with_prices.head()

,Item Name,Price,Purchase Count,Total Purchase Value
Item ID,,,,
0,Bone Crushing Silver Skewer,1.82,1,1.82
1,"Stormbringer, Dark Blade of Ending Misery",9.12,4,36.48
2,Primitive Blade,3.40,1,3.40
3,Final Critic,1.79,1,1.79
4,Stormfury Mace,2.28,1,2.28


In [204]:
#Rename the index
names_with_prices.index.names = ['Item ID']
#Sort by total number of purchases and print top five rows
names_with_prices.sort_values(['Purchase Count'], ascending=False).head(n=5)

,Item Name,Price,Purchase Count,Total Purchase Value
Item ID,,,,
39,Stormfury Mace,25.85,11,284.35
84,"Thorn, Satchel of Dark Souls",24.53,11,269.83
31,"Shadow Strike, Glory of Ending Hope",18.63,9,167.67
175,Retribution Axe,11.16,9,100.44
13,"Piety, Guardian of Riddles",13.41,9,120.69


In [184]:
#Most Profitable Items

In [202]:
#Sort previous table by total purchase value instead of purchase count and print top five rows
names_with_prices.sort_values(['Total Purchase Value'], ascending=False).head(n=5)

,Item Name,Price,Purchase Count,Total Purchase Value
Item ID,,,,
34,"Alpha, Reach of Ending Hope",37.26,9,335.34
39,Stormfury Mace,25.85,11,284.35
84,"Thorn, Satchel of Dark Souls",24.53,11,269.83
107,Spectral Diamond Doomblade,28.88,8,231.04
115,"Thorn, Conqueror of the Corrupted",29.75,7,208.25


Observable Trends:

1. A majority of the top-selling items by purchase count were above the average price. This could point to a trend of higher  priced items being in higher demand. The mean purchase price is $12.49.

2. This trend echoes the same pattern as the first. All five items with the highest total purchase value were priced well above the mean item price.

3. Interestingly, the players in each age group acted like a bell curve, peaking with 20-24 year olds and minimizing outward towards the youngest and oldest players. This pattern carries over into total purchase value, but not average purchase price or normalized totals. 
